In [1]:
!pip install demoji
!pip install emoji
!pip install pychecker
!pip install contractions
!pip install wordninja

ERROR: Could not find a version that satisfies the requirement pychecker (from versions: none)
ERROR: No matching distribution found for pychecker


* Lowercasing the data
* Fix Accented Words
* Expanding Contractions
* Removing emoticons
* Removing mentions
* Removing Hashtags
* Removing Numbers and Special Characters
* Removing extra space

In [1]:
import contractions
import demoji
import emoji
import nltk
import pandas as pd
import re
import unicodedata
import wordninja

demoji.download_codes()

C:\Users\rheyannmagcalas\AppData\Local\Temp\ipykernel_4372\540732830.py:10: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('twitter_disaster_tweet.csv')

In [4]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this # earthquake...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask . Canada,1
2,5,NaN,NaN,All residents asked to ' shelter in place ' ...,1
3,6,NaN,NaN,"13,000 people receive # wildfires evacuation ...",1
4,7,NaN,NaN,Just got sent this photo from Ruby # Alaska a...,1


In [5]:
#show feature data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


### References:
* https://machinelearningmastery.com/clean-text-machine-learning-python/
* https://towardsdatascience.com/nlp-building-text-cleanup-and-preprocessing-pipeline-eba4095245a0
* https://www.kaggle.com/takanorihasebe/text-cleaning-bert-and-transformer

### Converting to lowercase

In [5]:
data['text'] = data["text"].str.lower()

### Expanding Contractions
* Example: from “Y’all i’d contractions you’re expanded don’t think.” to you all I would contractions you are expanded do not think.

In [6]:
def expand_contractions(text):
    result = text
    try:
        result = contractions.text(text)
    except:
        pass
    
    contractions = {
        "ain't": "am not / are not",
        "aren't": "are not / am not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he had / he would",
        "he'd've": "he would have",
        "he'll": "he shall / he will",
        "he'll've": "he shall have / he will have",
        "he's": "he has / he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how has / how is",
        "i'd": "I had / I would",
        "i'd've": "I would have",
        "i'll": "I shall / I will",
        "i'll've": "I shall have / I will have",
        "i'm": "I am",
        "i've": "I have",
        "isn't": "is not",
        "it'd": "it had / it would",
        "it'd've": "it would have",
        "it'll": "it shall / it will",
        "it'll've": "it shall have / it will have",
        "it's": "it has / it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she had / she would",
        "she'd've": "she would have",
        "she'll": "she shall / she will",
        "she'll've": "she shall have / she will have",
        "she's": "she has / she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as / so is",
        "that'd": "that would / that had",
        "that'd've": "that would have",
        "that's": "that has / that is",
        "there'd": "there had / there would",
        "there'd've": "there would have",
        "there's": "there has / there is",
        "they'd": "they had / they would",
        "they'd've": "they would have",
        "they'll": "they shall / they will",
        "they'll've": "they shall have / they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we had / we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what shall / what will",
        "what'll've": "what shall have / what will have",
        "what're": "what are",
        "what's": "what has / what is",
        "what've": "what have",
        "when's": "when has / when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where has / where is",
        "where've": "where have",
        "who'll": "who shall / who will",
        "who'll've": "who shall have / who will have",
        "who's": "who has / who is",
        "who've": "who have",
        "why's": "why has / why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you had / you would",
        "you'd've": "you would have",
        "you'll": "you shall / you will",
        "you'll've": "you shall have / you will have",
        "you're": "you are",
        "you've": "you have"
    }
    
    for word in result.split():
        if word.lower() in contractions:
            result = result.replace(word, contractions[word.lower()])

    return result

In [7]:
data['text'] = data.apply(lambda x: expand_contractions(str(x['text'])),axis=1)

### Removing Accented Characters

* Example: 'Sómě Áccěntěd těxt. Some words such as résumé, café, prótest, divorcé, coördinate, exposé, latté.'

In [8]:
def remove_accented_chars(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

data['text'] = data.apply(lambda x: remove_accented_chars(str(x['text'])),axis=1)

### Emoji

In [9]:
def get_emoji_count(post):
    emoji = demoji.findall(post)
    name = [k for k,v in emoji.items()]
    return len(name)

data['emoji_count'] = data.apply(lambda x: get_emoji_count(str(x['text'])),axis=1)

In [10]:
def remove_emoji(text):
    return ''.join(c for c in str(text) if c not in emoji.EMOJI_DATA)

data['text'] = data.apply(lambda x: remove_emoji(x['text']),axis=1)

### Hashtag

In [17]:
import re

def get_hashtags(post):
    sample_result = re.findall("#(\s\w+)", post)
    
    return ','.join(sample_result), len(sample_result)

def get_hashtags_count(post):
    sample_result = re.findall("#(\s\w+)", post)
    
    return len(sample_result)

data['hashtags'] = data.apply(lambda x: get_hashtags(x['text']),axis=1)

In [18]:
data['text'] = data['text'].str.replace('[＃#](\s\w+)', ' ', case=False)

C:\Users\rheyannmagcalas\AppData\Local\Temp\ipykernel_4372\2332306136.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('[＃#](\s\w+)', ' ', case=False)


In [13]:
data.iloc[0]['text']

'our deeds are the reason of this    may allah forgive us all'

### Mentions

In [14]:
def get_mentions(post):
    sample_result = re.findall('@([a-zA-Z0-9]{1,15})', post)
    
    return sample_result

def get_mentions_count(post):
    sample_result = re.findall('@([a-zA-Z0-9]{1,15})', post)
    
    return len(sample_result)

data['mentions'] = data.apply(lambda x: get_mentions(x['text']),axis=1)

In [15]:
get_mentions('please help us @who @doh')

['who', 'doh']

In [16]:
data['text'] = data['text'].str.replace('@([a-zA-Z0-9]{1,15})', ' ', case=False)

C:\Users\rheyannmagcalas\AppData\Local\Temp\ipykernel_4372\4263372094.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('@([a-zA-Z0-9]{1,15})', ' ', case=False)


### Remove hyperlinks

In [19]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    text = str(text)
    if text != "":
        return TAG_RE.sub(' ', text)
    else:
        return text

data["text"] = data["text"].apply(remove_tags)

### Remove Numbers and Special Characters

In [20]:
def remove_numbers(text):
    pattern = r'[^a-zA-Z]' 
    return re.sub(pattern, ' ', text)

data['text'] = data.apply(lambda x: remove_numbers(str(x['text'])),axis=1)

### Remove Extra Whitespaces

In [21]:
def remove_extra_white_spaces(text):
    text = re.sub(' +', ' ', text)
    return text

data['text'] = data.apply(lambda x: remove_extra_white_spaces(str(x['text'])),axis=1)

In [22]:
data.to_csv('twitter_disaster_tweet_clean.csv')